In [1]:
import os

# os.environ["OPENAI_API_KEY"]

In [9]:
from langsmith import Client
client = Client()
project_runs = client.list_runs(project_name="pr-overcooked-worth-22")

In [4]:
!pip install langfree --quiet

In [11]:
i

<class 'langsmith.schemas.Run'>(id=ceb7f3ce-a67f-48b4-9264-a45bb1290445, name='ChatOpenAI', run_type='llm')

In [12]:
from langfree.runs import get_recent_runs
runs = get_recent_runs(last_n_days=3, limit=5)

Exception: You must set the environment variable LANGCHAIN_API_KEY

In [ ]:
from langfree.chatrecord import ChatRecordSet
llm_data = ChatRecordSet.from_runs(runs)

In [13]:
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"]="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"
os.environ["LANGSMITH_PROJECT"]="pr-overcooked-worth-22"

os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="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"
os.environ["LANGSMITH_PROJECT"]="pr-oavercooked-worth-22"


### Testing out the OpenAI model

In [3]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    messages=[{
        "role": "user",
        "content": "Say this is a test",
    }],
    model="gpt-4o-mini",
)

In [4]:
response

ChatCompletion(id='chatcmpl-AttEo6SAdUJdhf0EkEJeebm2CVSRT', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is a test.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1737883738, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_72ed7ab54c', usage=CompletionUsage(completion_tokens=6, prompt_tokens=12, total_tokens=18, completion_tokens_details=CompletionTokensDetails(audio_tokens=0, reasoning_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [28]:
# !pip install langchain langchain-openai

# Basic Tool Calling

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage


In [4]:
llm = ChatOpenAI(model="gpt-4o-mini")

### Defining Tools Using Functions

Tools can be defined using functions. By using a langchain decorator, or pydantic model, or from the BaseTool langchain class.

In [5]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.

@tool
def add(a: int, b: int) -> int:
    """Add two integers.
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers.
    """
    return a * b

In [6]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two integers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [7]:
multiply.args_schema.model_json_schema()

{'description': 'Multiply two integers.',
 'properties': {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'multiply',
 'type': 'object'}

In [8]:
multiply.invoke({"a":2, "b":3})

6

### Binding Tools to LLM and Calling

In [9]:
tools = [add, multiply]

In [10]:
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

message = llm_with_tools.invoke(query)
message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_GVIiZojA2g1614xJFqknEVIy', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 73, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_81274e7ad2', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-db81620f-f527-49c3-b26a-1fd5a0f10138-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_GVIiZojA2g1614xJFqknEVIy', 'type': 'tool_call'}], usage_metadata={'input_tokens': 73, 'output_tokens': 18, 'total_tokens': 91, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [11]:
message.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_GVIiZojA2g1614xJFqknEVIy',
  'type': 'tool_call'}]

### Multiple Tool Calls

In [12]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_EMmy4ZrWYtVl57H2hNE84dGW',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_SKi71mbxCQylVq32lDSqIG7w',
  'type': 'tool_call'}]

Optional: Pydantic output parsers, assuming the tool is defined using pydantic.

### Pass tool results back to Model

In [13]:
query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_LxytXaczgE4VfkuifofjU7Rf', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_W4PSWguamjusm6NQWL7VcId9', 'type': 'tool_call'}]


In [14]:
tools_dict = {"add": add, "multiply": multiply}

for tool_call in ai_msg.tool_calls:
    tool_output = tools_dict[tool_call["name"]].invoke(tool_call)
    messages.append(tool_output)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LxytXaczgE4VfkuifofjU7Rf', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_W4PSWguamjusm6NQWL7VcId9', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 83, 'total_tokens': 134, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dd0f1fd6-4636-4f50-aeda-49544991e0fc-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_Lxy

In [15]:
final_output = llm_with_tools.invoke(messages)

In [16]:
final_output.content

'The result of \\(3 \\times 12\\) is 36, and the result of \\(11 + 49\\) is 60.'

### More Tools

In [17]:
from collections import Counter

@tool
def character_count(string, char):
    """counts the number of given character in the given string
    """
    return Counter(string).get(char, 0)

In [18]:
character_count.invoke({"string":"strawberry", "char":"r"})

3

In [19]:
llm_with_tools = llm.bind_tools([add, multiply, character_count])

In [20]:
query = "How many 'w's are present in the word strawberry? Also what is 3+78"
messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)
print("#"*100)
print(ai_msg.tool_calls)
messages.append(ai_msg)

tools_dict = {"add": add, "multiply": multiply, "character_count": character_count}
print("#"*100)
for tool_call in ai_msg.tool_calls:
    tool_output = tools_dict[tool_call["name"]].invoke(tool_call)
    messages.append(tool_output)
final_output = llm_with_tools.invoke(messages)
final_output.content

####################################################################################################
[{'name': 'character_count', 'args': {'string': 'strawberry', 'char': 'w'}, 'id': 'call_2e5bXzCZHtjWC9wc46MwwBZB', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 3, 'b': 78}, 'id': 'call_34uswHF4PSwQFfcfHpyFz73j', 'type': 'tool_call'}]
####################################################################################################


'There is 1 \'w\' in the word "strawberry." Also, the sum of 3 and 78 is 81.'

### Tool-Ception

Nested query that requires you to pass output of function calls to another function call.

In while loop, keep performing the tool calls till you get an answer.

In [27]:
query = "What is the sum of the number of r's in strawberry and the number of b's in blueberry"
messages = [HumanMessage(query)]

output = llm_with_tools.invoke(messages)
messages.append(output)
while len(output.content) == 0 and len(output.tool_calls) !=0:
    for tool_call in output.tool_calls:
        tool_output = tools_dict[tool_call["name"]].invoke(tool_call)
        messages.append(tool_output)
    output = llm_with_tools.invoke(messages)
    messages.append(output)
output.content

'The sum of the number of \'r\'s in "strawberry" and the number of \'b\'s in "blueberry" is 5.'

In [49]:
# !pip install -U langsmith openai